In [109]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [110]:
#K-means
df = pd.read_excel("/content/drive/MyDrive/Practica 9/diabetes_prediction_dataset.xlsx").head(5000)

# Codificar las variables categóricas
df.dropna(subset=['gender', 'smoking_history'], inplace=True)

df['gender'] = df['gender'].map({'Male': 0, 'Female': 1, 'Other': 2})
df['smoking_history'] = df['smoking_history'].map({'never': 0, 'No Info': 1, 'current': 2, 'former': 3, 'ever': 4, 'not current': 5})

# Establecer la proporción de entrenamiento y prueba
proporcion_entrenamiento = 0.8
num_ejemplos = len(df)
num_ejemplos_entrenamiento = int(num_ejemplos * proporcion_entrenamiento)

# Dividir el conjunto de datos en entrenamiento y prueba
train = df[:num_ejemplos_entrenamiento]
test = df[num_ejemplos_entrenamiento:]

# Inicializar los pesos de manera aleatoria
np.random.seed(0)
weights = np.random.rand(train.shape[1])  # Incluyendo la columna "bias"

# Agregar una columna de unos para el sesgo (bias)
df["bias"] = 1

# Normaliza las características
mean = train.drop("diabetes", axis=1).mean()
std = train.drop("diabetes", axis=1).std()

train_features = (train.drop("diabetes", axis=1) - mean) / std
train_features = train_features.to_numpy()

test_features = (test.drop("diabetes", axis=1) - mean) / std
test_features = test_features.to_numpy()

In [111]:
# Función para calcular la distancia euclidiana entre dos puntos
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2)**2))

#Numero de clusters
k=3

# Inicializar los centroides aleatorios para K-Means
np.random.seed(0)
centroids = np.random.rand(k, train_features.shape[1])  # Se elige 2 como ejemplo, ajusta según tus necesidades

# Número máximo de iteraciones
max_iterations = 100

# Inicializar listas para los nuevos clústeres
new_clusters = [[] for _ in range(k)]  # Asegúrate de que new_clusters tenga la longitud correcta

# Asignar cada punto al clúster más cercano
for _ in range(max_iterations):
    # Inicializar listas para los nuevos clústeres
    new_clusters = [[] for _ in range(k)]  # Asegúrate de que new_clusters tenga la longitud correcta

    # Asignar cada punto al clúster más cercano
    for i, point in enumerate(train_features):
        distances = [euclidean_distance(point, centroid) for centroid in centroids]
        cluster_index = np.argmin(distances)
        new_clusters[cluster_index].append(i)

    # Calcular nuevos centroides
    for i in range(k):
        if len(new_clusters[i]) > 0:
            centroids[i] = np.mean(train_features[new_clusters[i]], axis=0)

# Asignar clústeres a los datos de prueba
predictions_kmeans = []
for point in test_features:
    distances = [euclidean_distance(point, centroid) for centroid in centroids]
    cluster_index = np.argmin(distances)
    predictions_kmeans.append(cluster_index)

In [117]:
# Comparar asignaciones de clústeres con etiquetas reales
y_test = test["diabetes"].to_numpy()
accuracy_kmeans = np.mean(predictions_kmeans == y_test)
print("Precisión del modelo K-Means:", accuracy_kmeans)

Precisión del modelo K-Means: 0.436


In [118]:
# Agregar la columna "Cluster" al DataFrame de prueba
test["Cluster"] = predictions_kmeans

# Mostrar el DataFrame con las asignaciones de clústeres
test

<ipython-input-118-b75dd68feb62>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["Cluster"] = predictions_kmeans


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes,Cluster
4000,0,23.0,0,0,2,27.32,6.0,159,0,1
4001,1,69.0,0,0,1,27.32,3.5,85,0,0
4002,1,73.0,1,0,0,27.80,6.8,280,1,2
4003,1,27.0,0,0,4,27.32,5.7,90,0,0
4004,0,28.0,0,0,2,26.70,4.8,160,0,1
...,...,...,...,...,...,...,...,...,...,...
4995,1,18.0,0,0,1,27.32,6.0,130,0,1
4996,0,79.0,0,0,5,25.77,6.0,130,1,0
4997,1,44.0,0,0,1,27.32,3.5,140,0,1
4998,1,61.0,1,0,0,33.11,5.0,130,0,2


In [ ]:
#KNN
df = pd.read_excel("/content/drive/MyDrive/Practica 9/diabetes_prediction_dataset.xlsx").head(5000)

# Codificar las variables categóricas
df.dropna(subset=['gender', 'smoking_history'], inplace=True)

df['gender'] = df['gender'].map({'Male': 0, 'Female': 1, 'Other': 2})
df['smoking_history'] = df['smoking_history'].map({'never': 0, 'No Info': 1, 'current': 2, 'former': 3, 'ever': 4, 'not current': 5})

# Establecer la proporción de entrenamiento y prueba
proporcion_entrenamiento = 0.8
num_ejemplos = len(df)
num_ejemplos_entrenamiento = int(num_ejemplos * proporcion_entrenamiento)

# Dividir el conjunto de datos en entrenamiento y prueba
train = df[:num_ejemplos_entrenamiento]
test = df[num_ejemplos_entrenamiento:]

# Inicializar los pesos de manera aleatoria
np.random.seed(0)
weights = np.random.rand(train.shape[1])  # Incluyendo la columna "bias"

# Agregar una columna de unos para el sesgo (bias)
df["bias"] = 1

# Calcular manualmente la media y la desviación estándar de las características de entrenamiento
mean = train.drop("diabetes", axis=1).mean()
std = train.drop("diabetes", axis=1).std()

# Aplicar la normalización a las características de entrenamiento
train_features = train.drop("diabetes", axis=1)
train_features = (train_features - mean) / std
train_features = train_features.to_numpy()

# Aplicar la normalización a las características de prueba
test_features = test.drop("diabetes", axis=1)
test_features = (test_features - mean) / std
test_features = test_features.to_numpy()

In [ ]:
# Función para calcular la distancia euclidiana entre dos puntos
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2)**2))

# Función de clasificador K-NN
def knn_predict(df_train, df_test, k):
    X_train = df_train.drop("diabetes", axis=1).to_numpy()
    y_train = df_train["diabetes"].to_numpy()

    X_test = df_test.drop("diabetes", axis=1).to_numpy()

    # Inicializar una lista para almacenar las predicciones
    y_pred = []

    for i in range(len(X_test)):
        # Calcular las distancias entre el punto de prueba y todos los puntos de entrenamiento
        distances = [euclidean_distance(X_test[i], x) for x in X_train]

        # Obtener los índices de los k vecinos más cercanos
        k_nearest_indices = np.argsort(distances)[:k]

        # Obtener las etiquetas de los k vecinos más cercanos
        k_nearest_labels = y_train[k_nearest_indices]

        # Realizar la predicción tomando la etiqueta más común entre los vecinos más cercanos
        prediction = np.bincount(k_nearest_labels).argmax()

        # Agregar la predicción a la lista
        y_pred.append(prediction)

    return np.array(y_pred)

# Función de error cuadrático medio
def compute_mean_squared_error(df, y_pred):
    y_true = df["diabetes"].to_numpy()
    mse = np.mean((y_true - y_pred)**2)
    return mse

In [ ]:
# Realizar predicciones en el conjunto de prueba
predictions = knn_predict(train, test, k=3)

# Evaluación del modelo (métricas de evaluación)
y_test = test["diabetes"].to_numpy()
accuracy = np.mean(predictions == y_test)
print("Precisión del modelo:", accuracy)

Precisión del modelo: 0.939


In [ ]:
# Agregar una columna "Prediction" al DataFrame de prueba
test["Prediction"] = predictions
# Mostrar el DataFrame con las clasificaciones
test

<ipython-input-52-b31b5fd91acc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["Prediction"] = predictions


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes,Prediction
4000,0,23.0,0,0,2,27.32,6.0,159,0,0
4001,1,69.0,0,0,1,27.32,3.5,85,0,0
4002,1,73.0,1,0,0,27.80,6.8,280,1,1
4003,1,27.0,0,0,4,27.32,5.7,90,0,0
4004,0,28.0,0,0,2,26.70,4.8,160,0,0
...,...,...,...,...,...,...,...,...,...,...
4995,1,18.0,0,0,1,27.32,6.0,130,0,0
4996,0,79.0,0,0,5,25.77,6.0,130,1,0
4997,1,44.0,0,0,1,27.32,3.5,140,0,0
4998,1,61.0,1,0,0,33.11,5.0,130,0,0
